In [ ]:
pip install selenium pandas


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

proxy_ip_port = '27.121.87.187:80'  

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'--proxy-server={proxy_ip_port}')

service = Service('C:\Dev\chrome-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)


Подгрузка необходимых библиотек. Настройка прокси, чтобы не активировалась защита на сайте. В качестве моста использую Хром-Селениум. 
(Прокси бесплатный, для проверки работоспособности вставьте свой)

In [7]:
#файл конфигурации выглядит таким образом
{
  "store": {
    "base_url": "https://sbermarket.ru/",
    "categories": [
      "Sales",
      "Beverages",
      "Vegetables"
    ]
  }
}


{'store': {'base_url': 'https://sbermarket.ru/',
  'categories': ['Sales', 'Beverages', 'Vegetables']}}

In [8]:
import json 
def load_configuration(config_path='config.json'):
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    return config['store']['base_url'], config['store']['categories']

store_base_url, categories = load_configuration()

print(f"Базовый URL магазина: {store_base_url}")
print(f"Категории для парсинга: {categories}")


Базовый URL магазина: https://sbermarket.ru/
Категории для парсинга: ['Sales', 'Beverages', 'Vegetables']


Использование конфигурации в скрипте

In [ ]:
def scrape_category_products(category):
    driver.get(f"{store_base_url}{category}")
    time.sleep(5)  

    products = []
    
    product_elements = driver.find_elements(By.CSS_SELECTOR, 'product-selector')  
    for product in product_elements:
        name = product.find_element(By.CSS_SELECTOR, 'name-selector').text  
        price = product.find_element(By.CSS_SELECTOR, 'price-selector').text  
        url = product.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        
        products.append({
            'category': category,
            'name': name,
            'price': price,
            'url': url
        })

    return products

all_products = []
for category in categories:
    all_products.extend(scrape_category_products(category))

df = pd.DataFrame(all_products)
df.to_csv('products.csv', index=False)


Извлечение маркированных данных и запись в csv для удобства